# Multi-outputs Task

In [ ]:
!pip install polars[pandas]

In [2]:
import numpy
import pandas # or use `polars`
import torch

## Data Processing from scikit-learn

In [3]:
from sklearn.datasets import make_multilabel_classification

In [4]:
X, y = make_multilabel_classification(n_samples=1000, 
                                      n_features=10,
                                      n_classes=3,
                                      n_labels=2,
                                      random_state=1)

In [5]:
X.shape, y.shape

((1000, 10), (1000, 3))

## Machine Learning Process

### Load Perming and Config Hyperparameters


In [6]:
import perming
main = perming.Box(10, 3, (30,), batch_size=16, activation='relu', inplace_on=True, solver='sgd', criterion="MultiLabelSoftMarginLoss", learning_rate_init=0.01)
# main = perming.Ranker(10, 3, (30,), batch_size=16, activation='relu', solver='sgd', criterion="MultiLabelSoftMarginLoss", learning_rate_init=0.01)
# main = perming.COMMON_MODELS['Multi-outputs'](10, 3, (30,), batch_size=16, activation='relu', solver='sgd', criterion="MultiLabelSoftMarginLoss", learning_rate_init=0.01)
main.print_config()

MLP(
  (mlp): Sequential(
    (Linear0): Linear(in_features=10, out_features=30, bias=True)
    (Activation0): ReLU(inplace=True)
    (Linear1): Linear(in_features=30, out_features=3, bias=True)
  )
)


OrderedDict([('torch -v', '1.7.1+cu101'),
             ('criterion', MultiLabelSoftMarginLoss()),
             ('batch_size', 16),
             ('solver',
              SGD (
              Parameter Group 0
                  dampening: 0
                  lr: 0.01
                  momentum: 0
                  nesterov: False
                  weight_decay: 0
              )),
             ('lr_scheduler', None),
             ('device', device(type='cuda'))])

### Dataloader from Numpy with Multi-threaded


In [7]:
main.data_loader(X, y, random_seed=0)

### Training Stage and Accelerated Validation


In [8]:
main.train_val(num_epochs=20, interval=10)

Epoch [1/20], Step [10/50], Training Loss: 0.5813, Validation Loss: 0.6255
Epoch [1/20], Step [20/50], Training Loss: 0.5474, Validation Loss: 0.5794
Epoch [1/20], Step [30/50], Training Loss: 0.5363, Validation Loss: 0.5585
Epoch [1/20], Step [40/50], Training Loss: 0.5204, Validation Loss: 0.5364
Epoch [1/20], Step [50/50], Training Loss: 0.4003, Validation Loss: 0.5185
Epoch [2/20], Step [10/50], Training Loss: 0.3562, Validation Loss: 0.5080
Epoch [2/20], Step [20/50], Training Loss: 0.4315, Validation Loss: 0.4914
Epoch [2/20], Step [30/50], Training Loss: 0.4022, Validation Loss: 0.4806
Epoch [2/20], Step [40/50], Training Loss: 0.4219, Validation Loss: 0.4698
Epoch [2/20], Step [50/50], Training Loss: 0.4118, Validation Loss: 0.4621
Epoch [3/20], Step [10/50], Training Loss: 0.4103, Validation Loss: 0.4508
Epoch [3/20], Step [20/50], Training Loss: 0.4358, Validation Loss: 0.4449
Epoch [3/20], Step [30/50], Training Loss: 0.4108, Validation Loss: 0.4341
Epoch [3/20], Step [40/50

### Test Model with Loss


In [9]:
main.test()

loss of Box on the 112 test dataset: 0.20931537449359894. accuracy: 0.0000 %


OrderedDict([('problem', 'multi-outputs'),
             ('loss',
              {'train': 0.25271689891815186,
               'val': 0.28921976685523987,
               'test': 0.20931537449359894})])

In [10]:
X, y = torch.as_tensor(X, dtype=torch.float).to(torch.device("cuda")), torch.as_tensor(y, dtype=torch.float).to(torch.device("cuda"))

In [11]:
pred = main.model(X) # predicted

In [12]:
'{}%'.format(sum(row.all().int().item() for row in (pred.ge(0.5) == y)) / X.shape[1])

'72.8%'

### Save Model Parameters to Models Folder


In [13]:
main.save(show=False, dir='../models/outputs.ckpt')

### Load Model Parameters from Models Folder

In [14]:
main.load(show=False, dir='../models/outputs.ckpt')